#Obtenção dos dados do CAGED

Este notebook trata da importação dos dados do CAGED (Cadastro Geral de Empregados e Desempregados), obtidos via FTP no endereço ftp://ftp.mtps.gov.br/pdet/microdados/CAGED, do antigo Ministério do Trabaho e Emprego (MTE). Esse sistema foi criado como registro permanente de admissões e dispensa de empregados, sob o regime da Consolidação das Leis do Trabalho (CLT), portanto trata de empregos formais.

As informações são prestadas pelos empregadores, de forma individual, para cada admissão/demissão, logo formam arquivos de dados bastante extensos, divididos por período (mês/ano).

Serão importados os dados referentes ao período de 2007 a 2018. O objetivo final é agrupar os dados por Ano, munícipio e as seguintes varíaveis de interesse: CBO (Classificação Brasileira de Ocupações), Tamanho do estabelecimento (Faixa de quantidada de empregados no início de janeiro daquele ano - Faixa Empr Início Jan), Grau de instrução, Subsetor Econômico segundo IBGE, raça/cor, sexo. Para cada agrupamento será totalizado o saldo de contratações do período e variação do total dos salários.

Quanto a Classe de Atividade Econômica (CNAE) optou-se por não utilizá-la, devido à possibilidade das pessoas jurídicas adotarem mais de um código CNAE, podendo ter prioridade em atividade econômica diversa daquela apontada como CNAE principal ou daquela informada ao CAGED, preferindo-se portanto utilizar o outro campo disponível (Subsetor Econômico segundo IBGE).

No presente notebook também são realizadas aglumas trasnformações de dados, como agrupamento e totalização, no intuito de reduzir o tamanho dos arquivos para poupar espaço e dividi-los em arquivos menores, para reduzir o consumo de memória nos processamentos posteriores a serem realizados em outros notebooks.

In [ ]:
from google.colab import drive
import os

# monta o drive no Google Drive para acesso aos dados
drive.mount('/content/drive/')

# Muda para a pasta de trabalho inicial onde estão os dados
os.chdir('/content/drive/My Drive/Colab Notebooks/TCC/CAGED/')

Mounted at /content/drive/


In [ ]:
# importação de bibliotecas
import pandas as pd
import numpy as np
! pip install py7zr # instala pacote seven zip
import py7zr as p7z

     |████████████████████████████████| 76 kB 2.7 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 131 kB 34.5 MB/s 
     |████████████████████████████████| 2.0 MB 59.5 MB/s 
     |████████████████████████████████| 357 kB 71.5 MB/s 
     |████████████████████████████████| 2.4 MB 48.3 MB/s 


In [ ]:
# abre a lista de maiores municípios, obtida na importação dos dados do PIB dos municípios
CodMaioresMunicipios = pd.read_excel(\
  '/content/drive/My Drive/Colab Notebooks/TCC/CodMaioresMunicipios.xlsx',squeeze=True)
# na base do CAGED o cód. do município não tem o último dígito
CodMaioresMunicipios = np.int64(CodMaioresMunicipios/10)
CodMaioresMunicipios 

array([355030, 330455, 530010, 310620, 410690, 130260, 431490, 353440,
       292740, 350950, 230440, 351880, 354870, 330100, 261160, 350570,
       520870, 352590, 354990, 330170, 355220, 150140, 317020, 354340,
       354780, 211130, 310670, 311860, 320530, 420910, 412550, 330330,
       500270, 353650, 430510, 353870, 354850, 240810, 510340, 290570,
       320500, 270430, 330240, 420820, 420540, 411370, 250750, 354880,
       221100, 355410, 430460, 150553, 330490, 330350, 354980, 420240,
       280030, 351380, 313670, 411520, 330070, 353060, 110020, 520110,
       355240, 352940, 260790, 330452, 410180, 350600, 351350, 330630,
       317010, 352050, 291080, 352690, 411990, 330390, 410830, 320520,
       351907, 350160, 351300, 350920, 430920, 313130, 520140, 355250,
       352440, 352730, 354890, 410480, 160030, 352250, 330010, 354390,
       421660, 351500, 330270, 350320, 260720, 351620, 431340, 420890,
       431560, 330510, 510760, 316720, 354730, 320130, 355280, 314480,
      

In [ ]:
# Acrescenta um período do CAGED (Ano/Mês) no dataset fornecido
def AcrescentarPeriodo (df, Ano, Mes):
  # nome do arquivo a ser descompactado e carregado
  arquivo = 'CAGEDEST_'+str(Mes).zfill(2)+str(Ano)
  # descompacta o arquivo do periodo que será acrescentado 
  with p7z.SevenZipFile(arquivo+'.7z', mode='r') as z:
    z.extractall()

  # ajusta o nome do arquivo conforme divergências encontradas nos dados
  if ((Ano == 2012) and (Mes == 6)):
    arquivo = 'CAGED_'+str(Mes).zfill(2)+str(Ano)
  elif ((Ano == 2015) and (Mes == 9)) or \
       ((Ano == 2015) and (Mes == 10)):
    arquivo = 'CAGEDEST'+str(Mes).zfill(2)+str(Ano)

  # carrega o arquivo do CAGED com as seguintes colunas: 
  # COMPETENCIA, MUNICIPIO, CBO 2002, IBGE subsetor, SALDO MOV
  periodo = pd.read_csv(arquivo+'.txt',sep=';',usecols=[1,2,4,8,9,11,15,16,17,18],encoding='latin_1',\
                 dtype={'Competência Declarada':int,'Município':int,'CBO 2002 Ocupação':str,\
                        'Faixa Empr Início Jan':str,'Grau Instrução':str,'IBGE Subsetor':str,\
                        'Raça Cor':str,'Sexo':str,'Salário Mensal':str,'Saldo Mov':int})
  # exclui o arquivo texto
  os.remove(arquivo + '.txt')
  # exclui cidades que não estão entre as 200 maiores em termos de PIB
  indexMenores = periodo.loc[~periodo['Município'].isin(CodMaioresMunicipios)].index
  periodo.drop(indexMenores, inplace=True)
  # converte o campo COMPETENCIA para ANO
  periodo.insert(2,'Ano',np.int64(periodo['Competência Declarada']/100))
  # exclui a coluna COMPETENCIA
  periodo.drop(columns=['Competência Declarada'],inplace=True)
  # renomeia a coluna Saldo Mov
  periodo.rename(columns={'Saldo Mov': 'SaldoMov'},inplace=True)
  # converte o campo Salário Mensal para apontar a variação positiva ou negativa
  periodo.insert(7,'Salário',periodo['Salário Mensal'].apply(lambda x: int(x[0:x.find(',')])))
  periodo['Salário'] = periodo['Salário'] * periodo['SaldoMov']
  # exclui a coluna Salário Mensal
  periodo.drop(columns=['Salário Mensal'],inplace=True)
  # limpa espaços dos campos que contenham string
  periodo = periodo.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
  
  # Adiciona o novo periodo ao dataframe
  return(df.append(periodo))

In [ ]:
# dataframes que vão acumular os saldos totais e por campos da variação de 
# salários e número de vagas de emprego

# total por município e ano
CAGED_MunicipiosSaldoTotal = pd.DataFrame()

# Totais por atributo de interesse
CAGED_MunicipiosSaldoCBO = pd.DataFrame()
CAGED_MunicipiosSaldoFaixaEmpr = pd.DataFrame()
CAGED_MunicipiosSaldoGrauInstrucao = pd.DataFrame()
CAGED_MunicipiosSaldoSubsetorEconomico = pd.DataFrame()
CAGED_MunicipiosSaldoRacaCor = pd.DataFrame()
CAGED_MunicipiosSaldoSexo = pd.DataFrame()

for Ano in range(2007,2019): # anos que serão analisados
  # dataframe para acumular os dados no ano que está sendo importado
  CAGED_MunicipiosTemp = pd.DataFrame()
  
  # Muda para a pasta de trabalho onde estão os dados do ano
  os.chdir('/content/drive/My Drive/Colab Notebooks/TCC/CAGED/'+str(Ano)+'/')

  for Mes in range(1,13): # os 12 meses que serão analisados
    CAGED_MunicipiosTemp = AcrescentarPeriodo(CAGED_MunicipiosTemp, Ano, Mes)
  
  # Acrescenta os dados do Ano totalizando as movimentações e o salário para 
  # obter os saldos agrupados pelos demais campos
  CAGED_MunicipiosSaldoTotal = \
    CAGED_MunicipiosSaldoTotal.append(CAGED_MunicipiosTemp.groupby(\
      ['Ano','Município'],as_index=False).aggregate(np.sum))

  CAGED_MunicipiosSaldoCBO = \
    CAGED_MunicipiosSaldoCBO.append(CAGED_MunicipiosTemp.groupby(\
      ['Ano','Município','CBO 2002 Ocupação'],as_index=False).aggregate(np.sum))
    
  CAGED_MunicipiosSaldoFaixaEmpr = \
    CAGED_MunicipiosSaldoFaixaEmpr.append(CAGED_MunicipiosTemp.groupby(\
      ['Ano','Município','Faixa Empr Início Jan'],as_index=False).aggregate(np.sum))

  CAGED_MunicipiosSaldoGrauInstrucao = \
    CAGED_MunicipiosSaldoGrauInstrucao.append(CAGED_MunicipiosTemp.groupby(\
      ['Ano','Município','Grau Instrução'],as_index=False).aggregate(np.sum))

  CAGED_MunicipiosSaldoSubsetorEconomico = \
    CAGED_MunicipiosSaldoSubsetorEconomico.append(CAGED_MunicipiosTemp.groupby(\
      ['Ano','Município','IBGE Subsetor'],as_index=False).aggregate(np.sum))

  CAGED_MunicipiosSaldoRacaCor = \
    CAGED_MunicipiosSaldoRacaCor.append(CAGED_MunicipiosTemp.groupby(\
      ['Ano','Município','Raça Cor'],as_index=False).aggregate(np.sum))
    
  CAGED_MunicipiosSaldoSexo = \
    CAGED_MunicipiosSaldoSexo.append(CAGED_MunicipiosTemp.groupby(\
      ['Ano','Município','Sexo'],as_index=False).aggregate(np.sum))

  # exclui as linhas com saldo de movimentação e salário zero
  CAGED_MunicipiosSaldoTotal.drop(\
    CAGED_MunicipiosSaldoTotal.loc[\
      (CAGED_MunicipiosSaldoTotal['Salário']==0)&
      (CAGED_MunicipiosSaldoTotal['SaldoMov']==0)].index,inplace=True)
  
  CAGED_MunicipiosSaldoCBO.drop(\
    CAGED_MunicipiosSaldoCBO.loc[\
      (CAGED_MunicipiosSaldoCBO['Salário']==0)&
      (CAGED_MunicipiosSaldoCBO['SaldoMov']==0)].index,inplace=True)
  
  CAGED_MunicipiosSaldoFaixaEmpr.drop(\
    CAGED_MunicipiosSaldoFaixaEmpr.loc[\
      (CAGED_MunicipiosSaldoFaixaEmpr['Salário']==0)&
      (CAGED_MunicipiosSaldoFaixaEmpr['SaldoMov']==0)].index,inplace=True)
  
  CAGED_MunicipiosSaldoGrauInstrucao.drop(\
    CAGED_MunicipiosSaldoGrauInstrucao.loc[\
      (CAGED_MunicipiosSaldoGrauInstrucao['Salário']==0)&
      (CAGED_MunicipiosSaldoGrauInstrucao['SaldoMov']==0)].index,inplace=True)
  
  CAGED_MunicipiosSaldoSubsetorEconomico.drop(\
    CAGED_MunicipiosSaldoSubsetorEconomico.loc[\
      (CAGED_MunicipiosSaldoSubsetorEconomico['Salário']==0)&
      (CAGED_MunicipiosSaldoSubsetorEconomico['SaldoMov']==0)].index,inplace=True)
  
  CAGED_MunicipiosSaldoRacaCor.drop(\
    CAGED_MunicipiosSaldoRacaCor.loc[\
      (CAGED_MunicipiosSaldoRacaCor['Salário']==0)&
      (CAGED_MunicipiosSaldoRacaCor['SaldoMov']==0)].index,inplace=True)
  
  CAGED_MunicipiosSaldoSexo.drop(\
    CAGED_MunicipiosSaldoSexo.loc[\
      (CAGED_MunicipiosSaldoSexo['Salário']==0)&
      (CAGED_MunicipiosSaldoSexo['SaldoMov']==0)].index,inplace=True)
  
# Volta para a pasta base do projeto
os.chdir('/content/drive/My Drive/Colab Notebooks/TCC/')

# salva os arquivos para uso futuro
CAGED_MunicipiosSaldoTotal.to_csv('CAGED_MunicipiosSaldoTotal.csv', index=False)
CAGED_MunicipiosSaldoCBO.to_csv('CAGED_MunicipiosSaldoCBO.csv', index=False)
CAGED_MunicipiosSaldoFaixaEmpr.to_csv('CAGED_MunicipiosSaldoFaixaEmpr.csv', index=False)
CAGED_MunicipiosSaldoGrauInstrucao.to_csv('CAGED_MunicipiosSaldoGrauInstrucao.csv', index=False)
CAGED_MunicipiosSaldoSubsetorEconomico.to_csv('CAGED_MunicipiosSaldoSubsetorEconomico.csv', index=False)
CAGED_MunicipiosSaldoRacaCor.to_csv('CAGED_MunicipiosSaldoRacaCor.csv', index=False)
CAGED_MunicipiosSaldoSexo.to_csv('CAGED_MunicipiosSaldoSexo.csv', index=False)

CAGED_MunicipiosSaldoTotal

,Ano,Município,Salário,SaldoMov
0,2007,110020,513380,2436
1,2007,120040,-105038,-219
2,2007,130260,4172492,21322
3,2007,140010,551247,1284
4,2007,150080,648108,2505
...,...,...,...,...
195,2018,520140,-3918290,1123
196,2018,520510,-823423,523
197,2018,520870,-26343070,5102
198,2018,521880,-2089987,989


In [ ]:
CAGED_MunicipiosSaldoSubsetorEconomico

,Ano,Município,IBGE Subsetor,Salário,SaldoMov
0,2007,110020,1,4440,4
1,2007,110020,10,-1842,8
2,2007,110020,11,1597,9
4,2007,110020,14,116114,163
5,2007,110020,15,390,107
...,...,...,...,...,...
4792,2018,530010,5,60616,11
4793,2018,530010,6,-131851,-37
4794,2018,530010,7,-138754,-22
4795,2018,530010,8,-356118,-45


In [ ]:
# Salva o arquivo códigos de município para uso futuro
os.chdir('/content/drive/My Drive/Colab Notebooks/TCC/')
pd.DataFrame(CodMaioresMunicipios,columns=['Código do Município']).to_excel('CodMaioresMunicipiosCAGED.xlsx', index=False)